### Deliverable 1 for 3 

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv('charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, concatenate
#!tf.keras --version

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# D3 do not drop name - include it as a feature, only drop EIN
application_df = application_df.drop(["EIN"], axis =1)
application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# drop non-usful columns ##D3
application_df = application_df.drop_duplicates()
application_df = application_df.dropna()
application_df.describe()

,STATUS,ASK_AMT,IS_SUCCESSFUL
count,22396.000000,2.239600e+04,22396.000000
mean,0.999777,4.238312e+06,0.651188
std,0.014940,1.077985e+08,0.476605
min,0.000000,5.000000e+03,0.000000
25%,1.000000,5.000000e+03,0.000000
50%,1.000000,5.000000e+03,1.000000
75%,1.000000,5.881325e+04,1.000000
max,1.000000,8.597806e+09,1.000000


In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [6]:
# Determine the number of unique values in each column.
application_df[application_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [7]:
# check counts in each category close to 10 unique values
app_count = application_df["APPLICATION_TYPE"].value_counts()
class_count = application_df["CLASSIFICATION"].value_counts()
#income_count = application_df["INCOME_AMT"].value_counts()
name_count = application_df["NAME"].value_counts()

In [9]:
#income_count

In [10]:
# # Trying Ordinal encoding for Income Amount
# from sklearn.preprocessing import OrdinalEncoder
# ord_list = ['0','1-9999','10000-24999','25000-99999','100000-499999', \
#            '1M-5M','5M-10M','10M-50M','50M+']
# ord_enc = OrdinalEncoder()
# ord_enc.fit(application_df["INCOME_AMT"].values.reshape(-1,1))

In [11]:
# ord_dict = {
#     '0':1,
#     '1-9999':2,
#     '10000-24999':3,
#     '25000-99999':4,
#     '100000-499999':5,
#     '1M-5M':6,
#     '5M-10M':7,
#     '10M-50M':8,
#     '50M+':9
# }

In [12]:
app_count

T3     18674
T6       935
T4       725
T7       515
T5       512
T19      344
T8       272
T9       156
T10      143
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [13]:
# Determine which values to replace
replace_apps = list(app_count[app_count < 500].index)
replace_class = list(class_count[class_count < 1000].index)
#replace_income = list(income_count[income_count < 500].index)
replace_name = list(name_count[name_count < 5].index)
# Replace in DataFrame
for i in replace_apps:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(i,"Other")
for i in replace_class:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(i,"Other")
#for i in replace_income:
   # application_df["INCOME_AMT"] = application_df["INCOME_AMT"].replace(i,"5M+")
for i in replace_name:
    application_df["NAME"] = application_df["NAME"].replace(i,"Other")

In [14]:
application_df.ASK_AMT

0            5000
1          108590
2            5000
3            6692
4          142590
           ...   
34290        5000
34293        5000
34294        5000
34297        5000
34298    36500179
Name: ASK_AMT, Length: 22396, dtype: int64

In [15]:
application_df["ASK_AMT"] = application_df["ASK_AMT"].astype(object)

In [16]:
application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,Other,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,Other,T5,CompanySponsored,Other,ProductDev,Association,1,0,N,5000,0
3,Other,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,Other,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34290,DISABLED AMERICAN VETERANS,T4,CompanySponsored,Other,Preservation,Association,1,1-9999,N,5000,0
34293,Other,T3,CompanySponsored,C1000,Preservation,Association,1,0,N,5000,1
34294,Other,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34297,Other,T5,Independent,Other,ProductDev,Association,1,0,N,5000,1


In [17]:

size_bins=[0,10000,20000,30000,40000,50000,60000,70000,
           80000,90000,100000,500000,1000000,5000000,10000000,
           100000000,10000000000
          ]
group_names=["5000-10000","10001-20000",'20001-30000',
             '30001-40000','40001-50000','50001-60000',
             '60001-70000','70001-80000','80001-90000',
             '90001-100000','100001-500000','500001-1000000',
             '1000001-5000000','5000001-10000000','10000001-100000000',
             '100000001+'
            ]
application_df["ASK_AMT"]=pd.cut(application_df["ASK_AMT"],size_bins,labels=group_names)
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,Other,Independent,C1000,ProductDev,Association,1,0,N,5000-10000,1
1,Other,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,100001-500000,1
2,Other,T5,CompanySponsored,Other,ProductDev,Association,1,0,N,5000-10000,0
3,Other,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,5000-10000,1
4,Other,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,100001-500000,1


In [18]:
application_df.dtypes

NAME                        object
APPLICATION_TYPE            object
AFFILIATION                 object
CLASSIFICATION              object
USE_CASE                    object
ORGANIZATION                object
STATUS                       int64
INCOME_AMT                  object
SPECIAL_CONSIDERATIONS      object
ASK_AMT                   category
IS_SUCCESSFUL                int64
dtype: object

In [19]:
application_df["ASK_AMT"] = application_df["ASK_AMT"].astype(object)

In [20]:
# Determine which values to replace if counts are less than ..?
# replace_class = list(classification_counts[classification_counts < 1883].index)

# # Replace in dataframe
# for cls in replace_class:
#     application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# # Check to make sure binning was successful
# application_df.CLASSIFICATION.value_counts()

In [21]:
application_df.dtypes

NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                   object
IS_SUCCESSFUL              int64
dtype: object

In [22]:
name_count

LITTLE LEAGUE BASEBALL INC                                     180
PARENT BOOSTER USA INC                                         174
HABITAT FOR HUMANITY INTERNATIONAL INC                         133
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                  112
UNITED STATES BOWLING CONGRESS INC                              97
                                                              ... 
NATIONAL SOCIETY OF COLONIAL DAMES                               1
BRYAN EDUCATION SUPPORT GROUP                                    1
ANTIOCH MISSIONARY BAPTIST CHURCH OF POMPANO INC                 1
SAN ANTONIO NEIGHBORHOOD ASSISTANCE CORP                         1
CENTRO LATINO SER-JOBS FOR PROGRESS A NONPROFIT CORPORATION      1
Name: NAME, Length: 19568, dtype: int64

In [23]:
ask_count = application_df["ASK_AMT"].value_counts()


In [24]:
ask_count

5000-10000            14044
100001-500000          2304
1000001-5000000         940
10001-20000             789
20001-30000             655
500001-1000000          650
30001-40000             523
40001-50000             431
50001-60000             399
10000001-100000000      311
60001-70000             296
70001-80000             271
80001-90000             251
5000001-10000000        225
90001-100000            206
100000001+              101
Name: ASK_AMT, dtype: int64

In [25]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Generate our categorical variable lists
app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AIR FORCE ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN INC,NAME_AMERICAN CHEMICAL SOCIETY,NAME_AMERICAN COLLEGE OF HEALTHCARE EXECUTIVES,...,ASK_AMT_30001-40000,ASK_AMT_40001-50000,ASK_AMT_5000-10000,ASK_AMT_5000001-10000000,ASK_AMT_500001-1000000,ASK_AMT_50001-60000,ASK_AMT_60001-70000,ASK_AMT_70001-80000,ASK_AMT_80001-90000,ASK_AMT_90001-100000
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(app_cat,1)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
num_input = number_input_features
branch1 =  num_input * 1.25
branch2 = 2

input_layer1 = num_input
input_layer2 = num_input/1.5
input_layer3 = num_input

inputs = Input(shape = X_train_scaled[0].shape)
branch_one = Dense(branch1,activation="relu")(inputs)
branch_two = Dense(branch2,activation="sigmoid")(inputs)

combine_branch = concatenate([branch_one,branch_two])
hidden3 = Dense(input_layer2,activation="relu")(combine_branch)

output_layer = Dense(1,activation="sigmoid")(hidden3)

nn = Model(inputs=inputs, outputs=output_layer)

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#change save_freq= 5
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=5,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
nn.save("AlphabetSoupCharity1.h5")

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model


nn.summary()

In [ ]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")